In [35]:
import pandas as pd
import twint
import numpy as np
import nest_asyncio
nest_asyncio.apply()
import datetime
import ast
from newspaper import Article
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))
import re

In [12]:
df = pd.read_csv("NepaliTimes_analysis.csv", parse_dates=['created_at'])

In [13]:
list(df)

['id',
 'created_at',
 'urls',
 'article',
 'level1',
 'level2',
 'level3',
 'level1_count',
 'level_2_3_valid',
 'level2_count',
 'level3_count']

In [14]:
def clean_text(raw):
    text = BeautifulSoup(raw, 'lxml').text
    word_tokens = word_tokenize(text.lower().rstrip())
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    return ' '.join(filtered_sentence)

In [15]:
def article(url):
    if (len(url)==0):
        return 404
    url = url[0]
    try:
        article = Article(url)
        article.download()
        article.parse()
        return clean_text(article.text)
    except Exception as e:
        return 404

In [16]:
def get_new_articles(newspaper, since, until):
    c = twint.Config()
    c.Username = newspaper
    c.Pandas = True
    c.Hide_output= True
    c.Since = since
    c.Until = until
    twint.run.Search(c)
    df = twint.storage.panda.Tweets_df
    df = df[['id', 'created_at', "urls"]]
    return df

In [33]:
data_level1 = [
    'arabl land',
    'avail data',
    'bureau statist',
    'busi survey',
    'cens publi',
    'cens pop',
    'children employ',
    'civil registr',
    'collect method',
    'commerci export',
    'complet rate',
    'consum electr',
    'consum energ',
    'data access',
    'data collect',
    'data compil',
    'data entri',
    'data manag',
    'data releas',
    'data standard',
    'data user',
    'demograph data',
    'densit popul',
    'develop data',
    'difusion dat',
    'direct statist',
    'disaggreg data',
    'electr access',
    'electr consumpt',
    'energi consumpt',
    'establish survey',
    'exchang rate',
    'extern debt',
    'fertil rate',
    'food import',
    'food product',
    'gender gap',
    'govern debt',
    'govern statist',
    'gross domest',
    'gross nation',
    'health expenditur',
    'health survey',
    'import marchandis',
    'improv data',
    'improv statist',
    'indic measur',
    'indic preci',
    'inflat rate',
    'institut statist',
    'interest payment',
    'intern tourism',
    'irrig land',
    'land use',
    'life expect',
    'livestock product',
    'merchandis export',
    'merchandis trade',
    'model statist',
    'mortal rate',
    'multilater debt',
    'nation account',
    'nation statist',
    'nation survey',
    'national brut',
    'national statist',
    'open data',
    'part revenus',
    'pay gap',
    'popul census',
    'popul growth',
    'popul rate',
    'price index',
    'produccion aliment',
    'purchas power',
    'qualiti data',
    'receit fiscal',
    'releas data',
    'revenu fiscal',
    'rural popul',
    'servic export',
    'statist agenc',
    'statist author',
    'statist avail',
    'statist committe',
    'statist data',
    'statist depart',
    'statist national',
    'statist offic',
    'statist servic',
    'statist studi',
    'survey catalogu',
    'tax payment',
    'tax revenu',
    'trade balanc',
    'unemploy rate',
    'use data',
    'water suppli',
    'youth unemploy',
    ]

data_level2 = [
    'accur',
    'adequ',
    'ambigu',
    'apropi',
    'bancal',
    'bias',
    'confiabl',
    'correct',
    'deceit',
    'deceiv',
    'decept',
    'defectu',
    'delud',
    'engan',
    'equivoc',
    'erreur',
    'erro',
    'erron',
    'errone',
    'error',
    'exact',
    'exat',
    'fake',
    'fallaci',
    'faux',
    'fiabl',
    'generaliz',
    'illus',
    'imparcial',
    'impartial',
    'imprecis',
    'improp',
    'inaccur',
    'incorrect',
    'inexact',
    'invalid',
    'limit',
    'manipul',
    'mislead',
    'mistaken',
    'parcial',
    'prec',
    'precis',
    'proper',
    'reliabl',
    'rigor',
    'rigour',
    'scientif',
    'spurious',
    'tromp',
    'trompeur',
    'unbias',
    'unreli',
    'unscientif',
    'unsound',
    'vag',
    'vagu',
    'val',
    'valid',
    ]

data_level3 = [
    'data manipul',
    'lead question',
    'manipul dat',
    'report bias',
    'sampl select',
    'sampl size',
    ]

data_level_indicator = [
    ' cpi ',
    ' fdi ',
    ' gdp ',
    ' gnp ',
    ' hdi ',
    ' wdi ',
    ]

filter_list = [' data ', ' record ', ' research ', ' statistics ',
               ' study ']

In [31]:
def level1_count(article):
    if article:
        article = str(article)
        keyword_list = []
        for word in data_level1:
            search_ = r"\b" + word.split()[0] + r"[a-zA-Z]*\s\b" \
                + word.split()[1] + r"[a-zA-Z]*"
            if re.search(search_, article):
                keyword_list.append(word)
        for word in data_level_indicator:
            if word in article:
                keyword_list.append(word)
        return keyword_list
    return []


def level2_count(article, valid):
    if valid == 1:
        article = str(article)
        keyword_list = []
        for word in data_level2:
            search_ = r"\b" + word + r"[a-zA-Z]*"
            if re.search(search_,  article):
                keyword_list.append(word)
        return keyword_list
    return []


def level3_count(article, valid):
    if valid == 1:
        article = str(article)
        keyword_list = []
        for word in data_level3:
            search_ = r"\b" + word + r"[a-zA-Z]*"
            if re.search(search_,   article):
                keyword_list.append(word)
        return keyword_list
    return []


def level_2_3_filter(article):
    article = str(article)
    for word in filter_list:
        if word in article:
            return 1
    return 0


def level_len(count_list):
    if type(count_list) == str:
        return (1 if len(count_list) > 2 else 0)
    return (1 if len(count_list) > 0 else 0)

In [17]:
df_ = get_new_articles('NepaliTimes', since=df.created_at.max().strftime("%Y-%m-%d %H:%M:%S"), until=datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.


In [18]:
df_['article'] = df_.urls.apply(article)

<ipython-input-18-fc832729a2b2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_['article'] = df_.urls.apply(article)


In [21]:
df_ = df_[df_['article']!=404]

In [25]:
df1 = df.append(df_)

In [26]:
df1

,id,created_at,urls,article,level1,level2,level3,level1_count,level_2_3_valid,level2_count,level3_count
0,1376906056824016911,2021-03-30 20:21:00+05:45,https://www.nepalitimes.com/banner/the-sun-ris...,"far-west neglected rest nepal centuries , brim...",[],[],[],0.0,0.0,0.0,0.0
1,1376875102919258113,2021-03-30 18:18:00+05:45,https://www.nepalitimes.com/opinion/educating-...,# metoo movement started 2006 activist sexual ...,[],[],[],0.0,0.0,0.0,0.0
2,1376843645316304902,2021-03-30 16:13:00+05:45,https://www.nepalitimes.com/latest/female-foot...,"monday , nepal clinched three nations cup defe...",[],[],[],0.0,1.0,0.0,0.0
3,1376810929963196416,2021-03-30 14:03:00+05:45,https://www.nepalitimes.com/editorial/devoluti...,one afternoon last month ramaroshan rural muni...,[],[],[],0.0,0.0,0.0,0.0
4,1376782744001253378,2021-03-30 12:11:00+05:45,https://www.nepalitimes.com/here-now/delayed-d...,"toiling 18 months qatar ‘ target earner ’ , ol...",[],[],[],0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
187,1377207976159715329,1.61719e+12,[https://www.nepalitimes.com/banner/tourism-th...,new breed social entrepreneurs trying new styl...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
188,1377175547428491265,1.61718e+12,[https://www.nepalitimes.com/latest/female-foo...,"monday , nepal clinched three nations cup defe...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
189,1377143150284693508,1.61717e+12,[https://www.nepalitimes.com/banner/everybody-...,nepal prepares 12th national population househ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
190,1377114086236770304,1.61716e+12,[https://www.nepalitimes.com/latest/nepal-rela...,"much criticism unnecessary hassles tourists , ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
df = df1.copy()

In [36]:
df['level1'] = df.apply(lambda x: (level1_count(x['article']) if pd.isnull(x.level1) else x.level1), axis=1)

In [38]:
df['level1_count'] = df.level1.apply(level_len)

In [39]:
df['level_2_3_valid'] = df['article'].apply(level_2_3_filter)

In [40]:
df['level2'] = df.apply(lambda x: (level2_count(x['article'
                                    ], x['level_2_3_valid'
                                    ]) if pd.isnull(x.level2) else x.level2),
                                    axis=1)


In [41]:
df['level2_count'] = df.level2.apply(level_len)

In [42]:
df['level3'] = df.apply(lambda x: (level3_count(x['article'
                                    ], x['level_2_3_valid'
                                    ]) if pd.isnull(x.level3) else x.level3),
                                    axis=1)

In [43]:
df['level3_count'] = df.level3.apply(level_len)

In [44]:
df.level1_count.sum()/df.shape[0]

0.06888812154696132

In [45]:
df.level2_count.sum()/df.level_2_3_valid.sum()

0.6492048643592142

In [46]:
df.level3_count.sum()/df.level_2_3_valid.sum()

0.0

In [47]:
df.to_csv('NepaliTimes_analysis.csv', index=False)